In [1]:
import numpy as np
import pandas as pd 
pd.set_option("max_columns",None)
from sklearn.linear_model import LinearRegression, Ridge
#from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.activations import relu,linear
#from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
#from matplotlib import units
import tensorflow as tf
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime
from sklearn.model_selection import StratifiedShuffleSplit, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error
from sklearn.inspection import plot_partial_dependence
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api  as sm
from scipy import stats
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.linear_model import Lasso, Ridge, SGDRegressor,LinearRegression,RidgeCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,RobustScaler
#from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb


pd.set_option('max_columns', 200)



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

DATA INITILAZING AND CLEANING

In [3]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

ONE HOT ENCODING

In [4]:
dataset= dataset.set_index(["Date"])
year = pd.DataFrame(data=pd.get_dummies(dataset.index.year,prefix="year"))
month = pd.DataFrame(data=pd.get_dummies(dataset.index.month, prefix="month"))
#day = pd.DataFrame(data=pd.get_dummies(dataset.index.day,prefix="day"))
hour = pd.DataFrame(data=pd.get_dummies(dataset.index.hour,prefix="hour"))
#frames = [year,month,day,hour]
frames = [year,month,hour]
onehot_encoded = ali.join(frames)



FEATURE ENGINEERING

In [5]:
onehot_encoded = onehot_encoded.drop(["Date","name","date","lat","lon"],axis=1)

NORMALIZATION OF FEATURES (EXCEPTIONS: PRODUCTION AND DATE TIME ENCODED FEATURES)

In [6]:
def normalize(df,column_list):
    for col in column_list:
        feature_range = (-1,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [7]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
normalized

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year_2020,year_2021,year_2022,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,-1.000000,-0.303571,-0.346,-1.000000,-1.000000,-1.000000,0.884332,-0.897959,-1.0,-0.312407,-0.989333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.000000,-0.339286,-0.518,-1.000000,-1.000000,-1.000000,0.949527,-0.816327,-1.0,-0.342302,-0.986667,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.000000,-0.367857,-0.650,-1.000000,-1.000000,-1.000000,0.970557,-0.795918,-1.0,-0.366218,-0.873333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.000000,-0.382143,-0.760,-1.000000,-1.000000,-1.000000,0.970557,-0.816327,-1.0,-0.378176,-0.733333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.000000,-0.435714,0.634,-1.000000,-1.000000,-1.000000,1.000000,-0.795918,-1.0,-0.423019,-0.576000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19031,-0.179179,-0.203571,0.644,0.146112,0.353916,-0.353991,-0.255521,-0.244898,-1.0,-0.121076,0.628000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19032,-1.000000,-0.160714,0.724,0.290626,0.440636,-0.229108,-0.312303,-0.285714,-1.0,-0.001495,0.698667,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
19033,-1.000000,-0.139286,0.930,0.051969,0.512826,-0.546714,-0.303891,-0.346939,-1.0,-0.028401,0.705333,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
19034,-1.000000,-0.150000,0.984,-0.277833,0.330760,-0.839906,-0.284963,-0.510204,-1.0,-0.082212,0.646667,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
target = normalized.loc[normalized.year_2021 == 1]
target = target.drop(["year_2020","year_2022","year_2021"],axis=1)
target = target.reset_index(drop=True)

In [ ]:
aw = normalized.loc[normalized.year_2022 == 1]
aw1 = aw.loc[aw.month_1 ==1]

#CHOOSİNG SPECIFIC TRAIN AND TEST DATA . I CHOOSE 1 YEAR LONG TRAIN DATA AND A WEEK OF TEST DATA 

In [108]:
train_one_year = normalized[8712:17133].reset_index(drop=True)
test_one_week = normalized[17133:17863].reset_index(drop=True)

In [109]:
train_one_year

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year_2020,year_2021,year_2022,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,-1.0,-0.332143,0.240,-1.0,-1.0,-1.0,0.482650,-0.571429,-1.0,-0.384155,-0.985333,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.0,-0.317857,-0.050,-1.0,-1.0,-1.0,0.421661,-0.653061,-1.0,-0.360239,-0.872000,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.0,-0.317857,0.008,-1.0,-1.0,-1.0,0.396425,-0.612245,-1.0,-0.369208,-0.741333,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.0,-0.357143,0.542,-1.0,-1.0,-1.0,0.581493,-0.510204,-1.0,-0.420030,-0.584000,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.0,-0.342857,0.536,-1.0,-1.0,-1.0,0.543638,-0.571429,-1.0,-0.399103,-0.425333,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8416,-1.0,-0.360714,0.000,-1.0,-1.0,-1.0,0.444795,-0.653061,-1.0,-0.402093,-0.252000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8417,-1.0,-0.346429,0.036,-1.0,-1.0,-1.0,0.375394,-0.795918,-1.0,-0.348281,-0.409333,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8418,-1.0,-0.350000,0.066,-1.0,-1.0,-1.0,0.379600,-0.816327,-1.0,-0.351271,-0.568000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8419,-1.0,-0.385714,0.388,-1.0,-1.0,-1.0,0.524711,-0.632653,-1.0,-0.426009,-0.725333,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [110]:
test_one_week

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year_2020,year_2021,year_2022,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,-0.967968,-0.342857,0.990,-0.845892,-0.664926,-0.992488,0.760252,-0.448980,-0.612766,-0.390135,0.264000,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.857858,-0.314286,1.000,-0.719768,-0.435187,-0.962911,0.699264,-0.469388,-0.714894,-0.339312,0.345333,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.805806,-0.292857,1.000,-0.657805,-0.320318,-0.949296,0.659306,-0.346939,-0.802128,-0.321375,0.389333,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,-0.723724,-0.285714,1.000,-0.669998,-0.334393,-0.956573,0.623554,-0.346939,-0.906383,-0.318386,0.392000,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,-0.647648,-0.282143,1.000,-0.707376,-0.401135,-0.965962,0.585699,-0.367347,-1.000000,-0.318386,0.350667,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,-1.000000,-0.425000,0.994,-1.000000,-1.000000,-1.000000,0.785489,-0.306122,-1.000000,-0.512706,-0.185333,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
726,-1.000000,-0.421429,0.986,-1.000000,-1.000000,-1.000000,0.766562,-0.448980,-1.000000,-0.494768,-0.344000,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
727,-1.000000,-0.421429,0.886,-1.000000,-1.000000,-1.000000,0.758149,-0.510204,-0.814894,-0.479821,-0.502667,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
728,-1.000000,-0.425000,0.938,-1.000000,-1.000000,-1.000000,0.785489,-0.489796,-0.768085,-0.488789,-0.657333,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
from keras.models import Sequential 
from tensorflow.keras import model_conf
from keras.layers import BatchNormalization, Dense, Add 
from keras.layers import Activation, Dropout, Flatten, Dense
def residual_block(x):
  x_skip = x
  x = Dense(model_conf['neuron_size'],activation='relu',kernel_regularizer=model_conf['regulizer'])(x)
  x = BatchNormalization()(x)
  x = Dropout(model_conf['dropout'])(x)

  x = Dense(model_conf['neuron_size'],activation='relu',kernel_regularizer=model_conf['regulizer'])(x)
  x = BatchNormalization()(x)
  x = Dropout(model_conf['dropout'])(x)

  adding = Add()([x_skip,x])
  return Activation('relu')(adding)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
sns.heatmap(onehot_encoded.iloc[:,:11].corr(), cmap = 'copper',annot = True)
plt.title('Heat Map for Correlations', fontsize = 20)
plt.show()

In [9]:

train = target[:7692]
test = target[7692:]

In [11]:

x_train = train.copy()
x_test = test.copy()

y_train = x_train.pop('Production')
y_test = x_test.pop('Production')

In [111]:
x_train = train_one_year.copy()
x_test = test_one_week.copy()

y_train = x_train.pop('Production')
y_test = x_test.pop('Production')

In [ ]:
y  = normalized["Production"]
x = normalized.drop("Production",axis=1)
#train test split

x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.8,shuffle=False,random_state=1)

In [ ]:
%%time
clfs = []
seed = 3

clfs.append(("LinearRegression", Pipeline([("LogReg", LinearRegression())])))

clfs.append(("XGB", Pipeline([("XGB", XGBRegressor())]))) 

clfs.append(("KNN", Pipeline([("KNN", KNeighborsRegressor())]))) 

clfs.append(("DTR", Pipeline([("DecisionTrees", DecisionTreeRegressor())]))) 

clfs.append(("RFRegressor", Pipeline([("RandomForest", RandomForestRegressor())]))) 

clfs.append(("GBRegressor", Pipeline([("GradientBoosting", GradientBoostingRegressor())]))) 

clfs.append(("MLP", Pipeline([("MLP Regressor", MLPRegressor())])))

clfs.append(("EXT Regressor",Pipeline([("ExtraTrees", ExtraTreeRegressor())])))

clfs.append(("SV Regressor", Pipeline([("ExtraTrees", SVR())])))

scoring = 'r2'
n_folds = 10
msgs = []
results, names  = [], [] 

for name, model  in clfs:
    kfold = KFold(n_splits=n_folds, random_state=None)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(), cv_results.std())
    msgs.append(msg)
    print(msg)

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1))


In [170]:
def build_and_compile_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  return model


In [164]:
ali = build_and_compile_model()
ali.fit(x_train,y_train,batch_size=64,epochs=200)

Epoch 1/200
121/121 [==============================] - 0s 1ms/step - loss: 0.1899 - mean_absolute_error: 0.1899
Epoch 2/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0773 - mean_absolute_error: 0.0773
Epoch 3/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0698 - mean_absolute_error: 0.0698
Epoch 4/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0654 - mean_absolute_error: 0.0654
Epoch 5/200
121/121 [==============================] - 0s 2ms/step - loss: 0.0634 - mean_absolute_error: 0.0634
Epoch 6/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0622 - mean_absolute_error: 0.0622
Epoch 7/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0612 - mean_absolute_error: 0.0612
Epoch 8/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0594 - mean_absolute_error: 0.0594
Epoch 9/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0588 - mean_absolute_error:

In [145]:

my_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16,activation="relu"),
    tf.keras.layers.Dense(units=8,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="linear")
])

my_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.MeanAbsoluteError()],
    loss='mse')

my_model.fit(x_train,y_train,batch_size=64,epochs=200)

Epoch 1/200
121/121 [==============================] - 1s 1ms/step - loss: 0.3163 - mean_absolute_error: 0.4520
Epoch 2/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0579 - mean_absolute_error: 0.1720
Epoch 3/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0391 - mean_absolute_error: 0.1273
Epoch 4/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0322 - mean_absolute_error: 0.1049
Epoch 5/200
121/121 [==============================] - 0s 1000us/step - loss: 0.0297 - mean_absolute_error: 0.0947
Epoch 6/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0285 - mean_absolute_error: 0.0903
Epoch 7/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0278 - mean_absolute_error: 0.0880
Epoch 8/200
121/121 [==============================] - 0s 1ms/step - loss: 0.0273 - mean_absolute_error: 0.0852
Epoch 9/200
121/121 [==============================] - 0s 992us/step - loss: 0.0269 - mean_absolute_e

In [160]:

exp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="linear")
])

exp_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.MeanAbsoluteError()],
    loss='mse')

exp_model.fit(x_train,y_train,batch_size=64,epochs=50)

Epoch 1/50
121/121 [==============================] - 0s 1ms/step - loss: 0.1230 - mean_absolute_error: 0.2616
Epoch 2/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0425 - mean_absolute_error: 0.1380
Epoch 3/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0342 - mean_absolute_error: 0.1135
Epoch 4/50
121/121 [==============================] - 0s 2ms/step - loss: 0.0312 - mean_absolute_error: 0.1026
Epoch 5/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0295 - mean_absolute_error: 0.0958
Epoch 6/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0284 - mean_absolute_error: 0.0901
Epoch 7/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0277 - mean_absolute_error: 0.0870
Epoch 8/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0272 - mean_absolute_error: 0.0846
Epoch 9/50
121/121 [==============================] - 0s 1ms/step - loss: 0.0268 - mean_absolute_error: 0.0823
E

In [171]:
reg = LinearRegression()
reg.fit(x_train, y_train)

reg_train_pred = reg.predict(x_train)
reg_test_pred = reg.predict(x_test)

metrics(y_train,reg_train_pred,y_test,reg_test_pred)


for train data, mean squared error is 0.0329
for train data, mean absolute error is: 0.10639
for train data, mean absolute percentage error is 0.52634 

for test data, mean squared error is 5.604015317551524e+22
for test data, mean absolute error is: 236728015189.40512
for test data, mean absolute percentage error is 838758352296.1788


In [51]:
xgb = XGBRegressor()
xgb.fit(x_train, y_train)

xgb_train_pred = xgb.predict(x_train)
xgb_test_pred = xgb.predict(x_test)

metrics(y_train,xgb_train_pred,y_test,xgb_test_pred)



for train data, mean squared error is 0.001585586188426985
for train data, mean absolute error is: 0.019847477151901972
for train data, mean absolute percentage error is 0.1543493285237681 

for test data, mean squared error is 0.03807303537587897
for test data, mean absolute error is: 0.08720837932325866
for test data, mean absolute percentage error is 0.585500759511294


In [55]:
KN = KNeighborsRegressor()
KN.fit(x_train, y_train)

KN_train_pred = KN.predict(x_train)
KN_test_pred = KN.predict(x_test)

metrics(y_train,KN_train_pred,y_test,KN_test_pred)


for train data, mean squared error is 0.02026280612948404
for train data, mean absolute error is: 0.055029027987946344
for train data, mean absolute percentage error is 0.40131186599042795 

for test data, mean squared error is 0.03330210121583113
for test data, mean absolute error is: 0.08678253012958088
for test data, mean absolute percentage error is 0.5076838732134478


In [23]:
DTree = DecisionTreeRegressor()
DTree.fit(x_train, y_train)

DTree_train_pred = DTree.predict(x_train)
DTree_test_pred = DTree.predict(x_test)

metrics(y_train,DTree_train_pred,y_test,DTree_test_pred)


for train data, mean squared error is 1.922925373491156e-35
for train data, mean absolute error is: 1.7320171991032084e-19
for train data, mean absolute percentage error is 1.7577437878881462e-19 

for test data, mean squared error is 0.06188115997239249
for test data, mean absolute error is: 0.10060815273435302
for test data, mean absolute percentage error is 0.7264340009546084


In [180]:
r_forest = RandomForestRegressor()
r_forest.fit(x_train, y_train)

r_forest_train_pred = r_forest.predict(x_train)
r_forest_test_pred = r_forest.predict(x_test)

metrics(y_train,r_forest_train_pred,y_test,r_forest_test_pred)


for train data, mean squared error is 0.0034070034
for train data, mean absolute error is: 0.0225874887
for train data, mean absolute percentage error is 0.14587 

for test data, mean squared error is 0.04225
for test data, mean absolute error is: 0.08847
for test data, mean absolute percentage error is 0.73682


####################################################################################

In [40]:

new_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1,activation='tanh')
  ])

new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()],
  loss='mse')

In [ ]:
new_model.fit(
    x_train,
    y_train,
epochs=100)

In [41]:
new_model.fit(
    x_test,
    y_test,
epochs=100)

Epoch 1/100
23/23 [==============================] - 1s 1ms/step - loss: 0.5707 - mean_absolute_error: 0.6980 - mean_squared_error: 0.5707 - mean_absolute_percentage_error: 115.3837
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.1548 - mean_absolute_error: 0.3346 - mean_squared_error: 0.1548 - mean_absolute_percentage_error: 162.9182
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0892 - mean_absolute_error: 0.2224 - mean_squared_error: 0.0892 - mean_absolute_percentage_error: 142.2835
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0419 - mean_absolute_error: 0.1616 - mean_squared_error: 0.0419 - mean_absolute_percentage_error: 63.1254
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0277 - mean_absolute_error: 0.1103 - mean_squared_error: 0.0277 - mean_absolute_percentage_error: 59.8912
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0233 - mean_absol

In [47]:
big_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1,activation='tanh')
  ])

big_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()],
  loss='mse')

In [48]:
big_model.fit(
    x_train,
    y_train,
epochs=100)

Epoch 1/100
241/241 [==============================] - 1s 2ms/step - loss: 0.0951 - mean_absolute_error: 0.1889 - mean_squared_error: 0.0951 - mean_absolute_percentage_error: 72.2896
Epoch 2/100
241/241 [==============================] - 0s 1ms/step - loss: 0.0277 - mean_absolute_error: 0.0806 - mean_squared_error: 0.0277 - mean_absolute_percentage_error: 52.0642
Epoch 3/100
241/241 [==============================] - 0s 1ms/step - loss: 0.0260 - mean_absolute_error: 0.0750 - mean_squared_error: 0.0260 - mean_absolute_percentage_error: 51.8241
Epoch 4/100
241/241 [==============================] - 0s 1ms/step - loss: 0.0252 - mean_absolute_error: 0.0711 - mean_squared_error: 0.0252 - mean_absolute_percentage_error: 47.7683
Epoch 5/100
241/241 [==============================] - 0s 1ms/step - loss: 0.0246 - mean_absolute_error: 0.0699 - mean_squared_error: 0.0246 - mean_absolute_percentage_error: 53.8652
Epoch 6/100
241/241 [==============================] - 0s 1ms/step - loss: 0.0245 - m

In [49]:
big_model.fit(
    x_test,
    y_test,
epochs=100)

Epoch 1/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0311 - mean_absolute_error: 0.0798 - mean_squared_error: 0.0311 - mean_absolute_percentage_error: 57.3595
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0190 - mean_absolute_error: 0.0625 - mean_squared_error: 0.0190 - mean_absolute_percentage_error: 36.0688
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0165 - mean_absolute_error: 0.0603 - mean_squared_error: 0.0165 - mean_absolute_percentage_error: 34.9843
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0142 - mean_absolute_error: 0.0533 - mean_squared_error: 0.0142 - mean_absolute_percentage_error: 33.1470
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0133 - mean_absolute_error: 0.0521 - mean_squared_error: 0.0133 - mean_absolute_percentage_error: 39.0304
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0119 - mean_absolute

In [60]:
reg_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(16, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(8, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(4, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(1,activation='tanh')
  ])

reg_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()],
  loss='mse')

